# Konvolutivne neuronske mreže

Slojevi konvolutivnih mreža:


1.   Konvolutivni sloj (engl. *Convolution layer*)
2.   Sloj sažimanja (engl. *Pooling layer*)
3.   Potpuno povezan sloj (engl. *Fully connected layer*)


Iako se konvolutivne neuronske mreže najčešće dovode u kontekst sa problemima računarske vizije, važno je napomeuti da pronalaze svoje primene i u drugim oblastima poput obrade prirodnog jezika, analize zvuka i druge.

## Konvolutivni sloj

Pre nego što objasnimo konvolutivni sloj, potrebno je da razumemo kovoluciju, po kojoj je ova vrsta neuronske mreže dobila ime.

### Konvolucija
Konvolucija je matematički operator koji od dve funkcije f i g, proizvodi treću koja predstavlja količinu preklapanja između okrenute i prevedene verzije funkcije g i funkcije f.
$$ (f * g)(x) = \int_{-\infty}^{\infty} f(t) \cdot g(x - t) \, dt $$

<div style="text-align:center">
   <img src="imgs/konvolucija.gif" style='width:90%; max-width:30rem'/>
</div>

### 2D konvolucija
<img style="float:right; max-width:550px" src="imgs/2d_konvolucija.gif" width="80%" />

Diskretni 2D operator konvolucije predstavlja skalarni proizvod konvolucionog kernela sa delom slike nad kojim se vrši konvolucija.


### Šta to predstavlja za slike?
Sobelov operator se primenjuje za detekciju ivica na fotografiji, pri čemu koristi konvoluciju dva kernela (filtera) - horizontalnog i vertikalnog.

<div style="text-align:center">
   <img src="imgs/sobel.png" style='width:90%; max-width:30rem'/>
</div>


Umesto da filtere ručno kreiramo kako bismo izvukli relevantne osobine slike, konvolutivni slojevi, tokom procesa obučavanja, uče koji su optimalni filteri. Izlaz konvolutivnog sloja je N mapa osobina (engl. *feature maps*), gde N predstavlja broj filtera koji konvolutivni sloj koristi.

<div style="text-align:center">
   <img src="imgs/konvolutivni_sloj.jpg" style='width:90%; max-width:50rem'/>
</div>

Nakon konvolutivnog sloja može postojati aktivaciona funkcija.

<div style="text-align:center">
   <img src="imgs/RELU.jpg" style='width:90%; max-width:30rem'/>
</div>

Od čega zavise dimenzije izlaza konvolutivnog sloja?

1.   Broja filtera
2.   Koraka pri procesu konvolucije (engl. *stride*)
3.   Upotrebe *padding*-a

### Korak konvolucije - Stride
Predstavlja korak kojim pomeramo kernel prilikom računanja skalaranog proizvoda. U primeru ispod vidimo da je korak 1 zato što se kernel pomera za 1 mesto nakon čega računa skalarni proizvod.


<img style="float:right; max-width:400px" src="imgs/zero_padding.gif" width="70%" />

### Padding

Proces konvolucije nije moguće izvršiti za ivične piksele, rezultat toga je smanjenje dimenzija slike i gubitak informacija. Upotreba *padding*-a rešava ovaj problem dodavanjem dodatnih piksela oko ivica ulazne slike pre primene konvolucije. Postoji više načina na osnovu kojih se određuju vrednosti dodatih piksela, jedan od njih je da su novododati pikseli sa vrednošću nula (engl. *zero-padding*).

Ukoliko je ulaz u konvolutivni sloj, slika dimenzija 100x100x3 (RGB slika) i ukoliko taj sloj poseduje 4 filtera dimenzija 5x5x3, pri čemu je korak konvolucije 1 i prisutne je upotreba *padding*-a. Koje su dimenzije izlaza?

<div style="text-align:center">
   <img src="imgs/konv_izlaz.png" style='width:90%; max-width:50rem'/>
</div>

### Vizualizacija obučenih slojeva

<div style="text-align:center">
   <img src="imgs/v.png" style='width:90%; max-width:50rem'/>
</div>

## Sloj sažimanja

<div style="text-align:center">
   <img src="imgs/pooling_sloj.jpg" style='width:90%; max-width:50rem'/>
</div>


<img style="float:right; max-width:450px" src="imgs/pooling.png" width="80%" />

Izvršava *downsampling* nad mapama osobina. Dodatno smanjuje broj parametara smanjivanjem veličine mapa osobina. U zavisnosti od funkcije koja se izvršava pri pomeraju filtera, razlikujemo:


1.   Average pooling - prosek vrednosti
2.   Max pooling - najveća vrednost

Korišćenjem *pooling* slojeva, mreža može postati invarijantna na određene transformacije (npr. rotaciju, translaciju...). Zašto? *Pooling* slojevi zadržavaju relevantne informacije, bez obzira na tačnu lokaciju na slici.


Ukoliko je slika 32x32x1 (*grayscale* slika) ulaz u ovaj sloj i ukoliko koristimo filter dimenzija 2x2, sa korakom 2, izlaz će rezultovati slikom 16x16x1 - 4 puta manje parametara!

## Potpuno povezan sloj
<div style="text-align:center">
   <img src="imgs/klasifikacioni.jpg" style='width:90%; max-width:50rem'/>
</div>

MLP - najobičniji višeslojni perceptron koji radi klasifikaciju

## Ilustrovan prikaz propagacije unapred

<div style="text-align:center">
   <img src="imgs/t.jpg" style='width:90%; max-width:50rem'/>
</div>

## Konvolutivne neuronske mreže u PyTorch-u

Preuzimanje skupa podataka

In [56]:
import torch
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformacije koje ce biti primenjene na slike
transform = torchvision.transforms.Compose([
    transforms.ToTensor(),
])
batch_size = 128

train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform);
test = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transform);

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

Kreiranje konvolutivne neuronske mreže

In [49]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.feature_extraction = nn.Sequential(
            #ulaz 28x28x1, izlaz 28x28x16
            nn.Conv2d(
                # slike su grayscale pa je broj kanala 1, da je slika RGB broj kanala bi bio 3
                in_channels=1,
                # broj izlaznih kanala, 16 filtera -> 16 mapa osobina (kanala)
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),
            # sloj sažimanja će promeniti prve dve dimenzije mapa osobina (visinu i širinu), ali ne i treću (broj kanala)
            #ulaz 28x28x16, izlaz 14x14x16
            nn.MaxPool2d(kernel_size=2),

            # nije potrebno navođenje naziva argumenata, kao u kodu iznad, moguće je samo proslediti njihove vrednosti
            #ulaz 14x14x16, izlaz 14x14x32
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            #ulaz 14x14x32, izlaz 7x7x32
            nn.MaxPool2d(2),
        )

        self.flatten = nn.Flatten()

        # potpuno povezan sloj
        self.classification_head = nn.Linear(7*7*32, 10)

    def forward(self, x):
        x = self.feature_extraction(x)
        # prebacivanje dimenzija izlaza feature_extraction sloja u (batch_size, 7*7*32) kako bi odgovaralo dimenzijama ulaza klasifikacionog sloja
        x = self.flatten(x)
        output = self.classification_head(x)
        return output

In [50]:
cnn = CNN()
cnn.to(device)

CNN(
  (feature_extraction): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (classification_head): Linear(in_features=1568, out_features=10, bias=True)
)

Definisanje funkcije gubitka

In [51]:
loss_func = nn.CrossEntropyLoss()

Definisanje optimajzera

In [52]:
from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.00001)

Obučavanje modela

In [53]:
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        loss = loss_func(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: { loss.item():.4f}')

Epoch [1/10], Loss: 2.0679
Epoch [2/10], Loss: 1.5410
Epoch [3/10], Loss: 1.0444
Epoch [4/10], Loss: 0.7276
Epoch [5/10], Loss: 0.5546
Epoch [6/10], Loss: 0.4508
Epoch [7/10], Loss: 0.4569
Epoch [8/10], Loss: 0.4835
Epoch [9/10], Loss: 0.3770
Epoch [10/10], Loss: 0.3100


Evaluacija modela

In [54]:
def evaluate():
    with torch.inference_mode():
      correct = 0
      total = 0
      for images, labels in train_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = cnn(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print(f'Accuracy of the network on the 50000 train images:{ 100 * correct / total} %')

evaluate()

Accuracy of the network on the 50000 train images:90.47166666666666 %


Prikaz klasifikacije  10 primera iz test skupa

In [55]:
sample = next(iter(test_loader))
imgs, lbls = sample
imgs, lbls = imgs.to(device), lbls.to(device)

test_output = cnn(imgs[:10])
pred_y = torch.argmax(test_output, 1).cpu().numpy().squeeze()
actual_number = lbls[:10].cpu().numpy()

print(f'Prediction number: {pred_y}')
print(f'Actual number: {actual_number}')

Prediction number: [7 2 1 0 4 1 4 9 5 9]
Actual number: [7 2 1 0 4 1 4 9 5 9]
